# fz-Moret Plugin Example Usage

This notebook demonstrates how to use the fz-Moret plugin for parametric MORET calculations.

You'll learn how to:
1. Install the Funz framework and fz-Moret plugin
2. Parse MORET input files to identify variables
3. Compile input files with specific parameter values
4. Run parametric studies
5. Analyze results

**Note:** This notebook assumes MORET is installed at `/opt/MORET/scripts/moret.py`. If your installation is elsewhere, update the path in `.fz/calculators/Moret.sh`.

## Setup

First, install the Funz framework:

In [ ]:
!pip install git+https://github.com/Funz/fz.git

## Install Plugin

Install the Moret plugin using fz.install():

In [ ]:
import fz
fz.install('Moret')
print('Moret plugin installed successfully!')

## Download Example Files

Download the example MORET input file from the repository:

In [ ]:
import os
import urllib.request

# Create directory
os.makedirs('Moret', exist_ok=True)

# Download example file
url = 'https://raw.githubusercontent.com/Funz/fz-Moret/main/examples/Moret/godiva.m5'
urllib.request.urlretrieve(url, 'Moret/godiva.m5')
print('Downloaded godiva.m5')

## Parse Input Variables

Use `fz.fzi()` to identify variables in the MORET input file:

In [ ]:
# Parse the input file to find variables
variables = fz.fzi('Moret/godiva.m5', 'Moret')
print("Variables found in godiva.m5:")
print(variables)

## Compile Input Files

Use `fz.fzc()` to compile an input file with specific parameter values:

In [ ]:
# Compile with specific parameter values
fz.fzc(
    'Moret/godiva.m5',
    {'radius': 8.5, 'u5': 5.0e-02},
    'Moret',
    output_dir='compiled'
)

# Display the compiled file (in subdirectory)
import glob
compiled_files = glob.glob('compiled/*/godiva.m5')
if compiled_files:
    with open(compiled_files[0], 'r') as f:
        content = f.read()
        # Show relevant sections
        for line in content.split('\n'):
            if 'TYPE 1 SPHE' in line or 'U235' in line:
                print(line)

## Run Basic Calculation

Use `fz.fzr()` to run a parametric study varying the sphere radius:

**Note:** This requires MORET to be installed. If MORET is not available, the calculation will fail.

In [ ]:
try:
    # Run parametric study with different radius values
    results = fz.fzr(
        'Moret/godiva.m5',
        {'radius': [8.0, 8.2, 8.4, 8.6, 8.8]},
        'Moret',
        calculators='localhost_Moret',
        results_dir='results_basic'
    )
    
    print("\nResults:")
    print(results)
except Exception as e:
    print(f"Error running calculation: {e}")
    print("Make sure MORET is installed at /opt/MORET/scripts/moret.py")

## Multi-Parameter Study

Run a more complex parametric study varying both radius and U235 enrichment:

In [ ]:
import numpy as np

try:
    # Define parameter ranges
    input_variables = {
        'radius': np.linspace(8.0, 9.0, 5),
        'u5': [4.0e-02, 4.5e-02, 5.0e-02]
    }
    
    # Run parametric study
    results = fz.fzr(
        'Moret/godiva.m5',
        input_variables,
        'Moret',
        calculators='localhost_Moret',
        results_dir='results_multi'
    )
    
    print("\nMulti-parameter results:")
    print(results)
except Exception as e:
    print(f"Error running calculation: {e}")
    print("This requires MORET to be properly installed.")

## Visualize Results

Plot the k-effective values as a function of sphere radius:

In [ ]:
import matplotlib.pyplot as plt

try:
    # Extract results for plotting
    if 'results' in locals():
        radius_values = results['radius']
        keff_values = results['mean_keff']
        sigma_values = results['sigma_keff']
        
        # Create plot
        plt.figure(figsize=(10, 6))
        plt.errorbar(radius_values, keff_values, yerr=sigma_values, 
                     marker='o', linestyle='-', capsize=5)
        plt.xlabel('Sphere Radius (cm)')
        plt.ylabel('k-effective')
        plt.title('GODIVA Critical Assembly - k-eff vs Radius')
        plt.grid(True, alpha=0.3)
        plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Critical')
        plt.legend()
        plt.show()
    else:
        print("No results available to plot. Run the calculation cells first.")
except Exception as e:
    print(f"Error creating plot: {e}")